In [13]:
! git clone "https://github.com/gdbwoo/DS-3001-Projects"

fatal: destination path 'DS-3001-Projects' already exists and is not an empty directory.


In [4]:
! pip install mapclassify

In [5]:
# Import all necessary packages
import geopandas as gpd
import mapclassify
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [7]:
# Load data
#gdf = gpd.read_file('DS-3001-Projects/Project 3/VirginiaCounty_ClippedToShoreline.shx') # Load Virginia shapefile for maps (fix this)
gdf = gpd.read_file()
vdf = pd.read_csv('DS-3001-Projects/Project 3/voting_VA.csv') # Load historical voting data
cdf = pd.read_csv('DS-3001-Projects/Project 3/county_adjacencies.csv') # Load county adjacency data and population data

In [ ]:
# # Merge shapefile data and county adjacency data:
# gdf['FIPS_left'] = pd.to_numeric(gdf['STCOFIPS'])
# df = gdf.merge(cdf,left_on='FIPS_left',right_on='FIPS')  # merge county population and adjacency data into df
# del gdf, cdf

In [10]:
vdf.head()
vdf.shape
vdf = vdf.dropna()
vdf.shape()

(3736, 13)